In [31]:
import numpy as np
import pandas as pd
import subprocess
import requests
from time import sleep

In [32]:
def policy(pod, latency):
    return pod + latency

In [75]:
'''
build_table(3, 5, 2000, 5)


pod initial_latency	end_latency	      3     4	    5
  3               0	       2000    2003  2004	 2005
  4               0	       2000    2004  2005	 2006
  5               0	       2000    2005  2006	 2007
  3            2001	       4000    4003  4004	 4005
  4            2001	       4000    4004  4005	 4006
  5            2001	       4000    4005  4006	 4007
  3            4001	       8000    8003  8004	 8005
  4            4001	       8000    8004  8005	 8006
  5            4001	       8000    8005  8006	 8007
  3            8001	      16000   16003 16004	16005
  4            8001	      16000   16004 16005	16006
  5            8001	      16000   16005 16006	16007
  3           16001	      32000   32003 32004	32005
  4           16001	      32000   32004 32005	32006
  5           16001	      32000   32005 32006	32007
'''
def build_table(min_pods, max_pods, initial_lat, interval_size):
    intervals = []
    intervals.append((0, initial_lat))
    for x in range(interval_size - 1):
        initial_interval_value = intervals[-1][1]
        interval_value = intervals[0][1]
        # To use double value
        # last_interval_value = initial_interval_value * 2 
        last_interval_value = initial_interval_value + interval_value
        intervals.append((initial_interval_value + 1, last_interval_value))

    table = []

    for y in intervals:
        for x in range(min_pods, max_pods + 1):
            options = np.zeros(max_pods + 1 - min_pods)
            options = [policy(z + x, y[1]) for z in range(len(options))]
            
            table.append([x, y[0], y[1]] + list(options))

    labels = ['pod', 'initial_latency', 'end_latency']
    actions = list(np.arange(min_pods, max_pods + 1).astype(np.str_))

    return pd.DataFrame(table, columns=labels+actions)

In [63]:
'''
find_option(table, 5, 3000)

   3	    4	    5
2003	 2004	 2005
2004	 2005	 2006
2005	 2006	 2007
'''
def find_options(table, pod, latency):
    return table[(table['pod'] == pod) & (table['initial_latency'] <= latency) & (table['end_latency'] >= latency)].iloc[:,3::]

In [64]:
'''
find_best_action(table, 5, 3000)

3
'''
def find_best_action(table, pod, latency):
    options = find_options(table, pod, latency)
    
    return int(np.min(options).head(1).index.item())

In [65]:
def update_action_result(table, pod, latency, action, result):
        table.loc[(table['pod'] == pod) & (table['initial_latency'] <= latency) & (table['end_latency'] >= latency), str(action)] = result
        print("updating pod", pod, " latency ", latency, " result ", result)
        return table

In [66]:
def get_ip():
    # bashCommand = "kubectl get pods --field-selector=status.phase=Running"
    
    error = True
    output = None
    
    while(error == True):
        process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
        output, error = process.communicate()
    
    return output.decode("utf-8").split('\n')[1:][:-1].pop().split('   ')[3]

In [67]:
'''
return ms latency
'''
def get_latency():
    time = '1h'
    query = 'rate(http_server_request_duration_seconds_sum{path="/test"}[5m])/rate(http_server_request_duration_seconds_count{path="/test"}[5m])'
    result = None

    while(True):
        response = requests.get("http://localhost:9090/api/v1/query?query={query}".format(query = query))
        if response.json()['status'] == 'success':
            result = response.json()['data']['result']
            if result == []:
                result = float(0)
            else:
                result = response.json()['data']['result'][0]['value'][1]
                if result == 'NaN': result = float(0)
            break
        sleep(5)
    
    return float(result) * 1000

In [68]:
def get_pods():
    # TODO comando pega quando esta como Terminating
    bashCommand = "kubectl get pods --field-selector=status.phase=Running"
    
    error = True
    output = None
    
    while(error == True):
        process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
        output, error = process.communicate()
    
    return len(output.decode("utf-8").split('\n')[1:][:-1])  
    

In [69]:
def set_pods(new_pods):
    bashCommand = "kubectl scale deployment.v1.apps/phpa-web-app-deployment --replicas={pods}".format(pods = new_pods)

    error = True
    output = None

    while(error == True):
        process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
        output, error = process.communicate()

In [70]:
def run():
    for x in range(10):
        latency = get_latency()
        pods = get_pods()

        result = policy(pods, latency)

        print("\n--- Exec ", x)
        print("latency ", latency)
        print("pods ", pods)
        print("result ", result)

        if np.random.random() > epsilon:
            action = find_best_action(table, pods, latency)
            print("choose action ", action)
        else:
            action = np.random.randint(min_pods, max_pods + 1)
            print("random ", action)

        set_pods(action)
        sleep(75)

        new_latency = get_latency()
        new_result = policy(action, new_latency)

        print("new_latency ", new_latency)
        print("new_result ", new_result)        

        table = update_action_result(table, pods, latency, action, new_result)

        if new_result <= result:
            reward += 5
            print("reward ", reward)
        else:
            reward -= 5
            print("reward ", reward)

In [83]:
min_pods = 1
max_pods = 3
initial_latency = 500
interval_size = 12
reward = 0

epsilon = 0.5

In [84]:
table = build_table(min_pods, max_pods, initial_latency, interval_size)
table

,pod,initial_latency,end_latency,1,2,3
0,1,0,500,501,502,503
1,2,0,500,502,503,504
2,3,0,500,503,504,505
3,1,501,1000,1001,1002,1003
4,2,501,1000,1002,1003,1004
5,3,501,1000,1003,1004,1005
6,1,1001,1500,1501,1502,1503
7,2,1001,1500,1502,1503,1504
8,3,1001,1500,1503,1504,1505
9,1,1501,2000,2001,2002,2003
